Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Sreya Dhar"
COLLABORATORS = ""

---

In [2]:
from IPython.display import display, HTML
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows


In [3]:
conn = create_connection('non_normalized.db')
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)
display(df)

,StudentID,Name,Degree,Exams,Scores
0,1,"Rodriguez, Pamela",graduate,"exam7 (2017), exam9 (2018), exam3 (2018), exam...","61, 38, 85, 70, 44, 43, 68"
1,2,"Jackson, Kristie",undergraduate,"exam2 (2017), exam2 (2017), exam8 (2018), exam...","66, 69, 85, 62, 92, 72, 44, 50, 23"
2,3,"Curtis, George",graduate,"exam7 (2017), exam6 (2017), exam2 (2017), exam...","68, 73, 62, 72"
3,4,"Casey, Christie",undergraduate,"exam10 (2019), exam4 (2019), exam4 (2019), exa...","41, 52, 52, 70"
4,5,"Yoder, Emily",undergraduate,"exam4 (2019), exam4 (2019), exam4 (2019)","52, 45, 61"
...,...,...,...,...,...
95,96,"Dyer, Benjamin",undergraduate,"exam10 (2019), exam4 (2019), exam6 (2017), exa...","47, 53, 64, 57, 71, 79, 77, 56"
96,97,"Delgado, Jason",undergraduate,"exam1 (2016), exam8 (2018)","72, 83"
97,98,"Cortez, Kenneth",undergraduate,"exam3 (2018), exam5 (2020), exam6 (2017)","83, 55, 71"
98,99,"Short, Elizabeth",undergraduate,"exam3 (2018), exam9 (2018), exam3 (2018), exam...","87, 34, 84, 75, 80, 60, 48, 55, 75"


In [4]:
def fetch_degree_tables():
    sql_statement= "Select distinct(degree) from Students;"
    return execute_sql_statement(sql_statement, conn)

def fetch_exams_tables():
    sql_statement = "select Exams from Students;"
    out_exam = execute_sql_statement(sql_statement, conn)
    list_exam = []
    for ele in out_exam:
        for value in ele:
            values = value.split(",")
            for exam_year in values:
                list_exam.append(exam_year.strip())
    list_exam = list(dict.fromkeys(list_exam))
    exam_year=[]
    for ele in list_exam:
        exam, year = ele.split(' ')
        exam_year.append((exam, year[1:5]))
    return exam_year

def fetch_students_tables():
    sql_statement= "Select StudentID, name, degree from Students;"
    out_stud = execute_sql_statement(sql_statement, conn)
    stud_list = []
    for ele in out_stud:
        last_name, first_name = ele[1].split(',')
        stud_list.append((ele[0],first_name.strip(), last_name.strip(), ele[2]))
    return stud_list

def fetch_students_exam_score_tables():
    sql_statement = "Select StudentID, Exams, Scores from Students;"
    out_score = execute_sql_statement(sql_statement, conn)
    score_list = []
    for ele in out_score:
        exams, scores= ele[1].split(','), ele[2].split(',')
        for x in range(len(exams)):
            score_list.append((ele[0], exams[x].strip().split(' ')[0], scores[x].strip()))
    return score_list


def insert_exams(conn, values):
    sql_statement= """Insert into Exams(Exam, Year) values (?, ?);"""
    cur=conn.cursor()
    cur.execute(sql_statement, values)
    return cur.lastrowid

def insert_degree(conn, values):
    sql_statement = "INSERT INTO Degrees(degree) VALUES(?);"
    cur=conn.cursor()
    cur.execute(sql_statement, values)
    return cur.lastrowid

def insert_students(conn, values):
    sql_statement = """insert into Students (StudentID, First_Name, Last_Name, Degree) Values(?, ?, ?, ?);"""
    cur=conn.cursor()
    cur.execute(sql_statement, values)
    return cur.lastrowid

def insert_students_exam_score(conn, values):
    sql_statement= """insert into StudentExamScores(StudentID, Exam, Score) Values(?, ?, ?);"""
    cur=conn.cursor()
    cur.execute(sql_statement, values)
    return cur.lastrowid

def normalize_database(non_normalized_db_filename):
#     Normalize 'non_normalized.db'
#     Call the normalized database 'normalized.db'
#     Function Output: No outputs
#     Requirements:
#     Create four tables
#     Degrees table has one column:
#         [Degree] column is the primary key
    
#     Exams table has two columns:
#         [Exam] column is the primary key column
#         [Year] column stores the exam year
    
#     Students table has four columns:
#         [StudentID] primary key column 
#         [First_Name] stores first name
#         [Last_Name] stores last name
#         [Degree] foreign key to Degrees table
        
#     StudentExamScores table has four columns:
#         [PK] primary key column,
#         [StudentID] foreign key to Students table,
#         [Exam] foreign key to Exams table ,
#         [Score] exam score


    # YOUR CODE HERE
    create_deg_tab_sql= """create table [Degrees] ([Degree] TEXT NOT NULL PRIMARY KEY);"""
    
    create_exam_tab_sql = """create table [Exams] ([Exam] TEXT NOT NULL PRIMARY KEY,
                                                   [Year] INTEGER NOT NULL);"""
    
    create_stud_tab_sql = """create table [Students] ([StudentID] INTEGER NOT NULL PRIMARY KEY,
                                                      [First_Name] TEXT NOT NULL,
                                                      [Last_Name] TEXT NOT NULL,
                                                      [Degree] TEXT NOT NULL,
                                                      Foreign key (Degree) references Degrees(Degree));"""
                                                      
    
    create_score_tab_sql = """create table [StudentExamScores] ([PK] INTEGER NOT NULL PRIMARY KEY autoincrement,
                                                                [StudentID] INTEGER NOT NULL,
                                                                [Exam] TEXT NOT NULL,
                                                                [Score] REAL NOT NULL,
                                                                Foreign key (StudentID) references Students(StudentID),
                                                                Foreign key (Exam) references Exams(Exam));"""
    
    degrees = fetch_degree_tables()
    exams = fetch_exams_tables()
    students = fetch_students_tables()
    students_exam_score = fetch_students_exam_score_tables()
    db_file = 'normalized.db'
    normalize_conn = create_connection(db_file, True)
    
    with normalize_conn:
        create_table(normalize_conn, create_deg_tab_sql)
        create_table(normalize_conn, create_exam_tab_sql)
        create_table(normalize_conn, create_stud_tab_sql)
        create_table(normalize_conn, create_score_tab_sql)
        
        for ele in exams:
            insert_exams(normalize_conn, ele)
        for ele in degrees:
            insert_degree(normalize_conn, ele)
        for ele in students:
            insert_students(normalize_conn, ele)
        for ele in students_exam_score:
            insert_students_exam_score(normalize_conn, ele)

In [5]:
normalize_database('non_normalized.db')
conn = create_connection('normalized.db')


In [6]:
def ex1():
    # Write an SQL statement that SELECTs all rows from the `Exams` table and sort the exams by Year
    # output columns: exam, year
    
    # YOUR CODE HERE
    sql_statement= "SELECT exam, year FROM Exams ORDER BY Year, exam;"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
#     data = pd.read_csv("ex1.csv")
#     display(data)
    return sql_statement

In [7]:
sql_statement = ex1()
data = pd.read_csv("ex1.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Exam,Year
0,exam1,2016
1,exam2,2017
2,exam6,2017
3,exam7,2017
4,exam3,2018
5,exam8,2018
6,exam9,2018
7,exam10,2019
8,exam4,2019
9,exam5,2020


In [8]:
def ex2():
    # Write an SQL statement that SELECTs all rows from the `Degrees` table and sort the degrees by name
    # output columns: degree
    
    # YOUR CODE HERE
    sql_statement = "SELECT * FROM Degrees ORDER BY degree"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [9]:
sql_statement = ex2()
data = pd.read_csv("ex2.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Degree
0,graduate
1,undergraduate


In [10]:
def ex3():
    # Write an SQL statement that counts the numbers of gradate and undergraduate students
    # output columns: degree, count_degree
    
    # YOUR CODE HERE
    sql_statement="SELECT degree, COUNT(degree) as count_degree FROM Students group by degree;"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [11]:
sql_statement = ex3()
data = pd.read_csv("ex3.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Degree,count_degree
0,graduate,35
1,undergraduate,65


In [12]:
def ex4():
    # Write an SQL statement that calculates the exam averages for exams; sort by average in descending order.
    # output columns: exam, year, average
    # round to two decimal places
    
    # YOUR CODE HERE
    sql_statement="select StudentExamScores.exam, exams.year, round(avg(StudentExamScores.score),2) as average from 'StudentExamScores' inner join exams on StudentExamScores.exam =exams.exam group by StudentExamScores.exam order by average desc;"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [13]:
sql_statement = ex4()
data = pd.read_csv("ex4.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Exam,Year,average
0,exam3,2018,86.16
1,exam8,2018,80.18
2,exam1,2016,75.09
3,exam7,2017,68.96
4,exam2,2017,64.28
5,exam6,2017,61.80
6,exam4,2019,54.04
7,exam5,2020,45.56
8,exam10,2019,44.68
9,exam9,2018,39.43


In [14]:
def ex5():
    # Write an SQL statement that calculates the exam averages for degrees; sort by average in descending order.
    # output columns: degree, average 
    # round to two decimal places
    
    # YOUR CODE HERE
    sql_statement = "select students.degree, round(avg(StudentExamScores.score), 2) as average from StudentExamScores inner join students on StudentExamScores.studentID =students.studentID group by degree order by average desc;"
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
#     data = pd.read_csv("ex5.csv")
#     display(data)
    return sql_statement

In [15]:
sql_statement = ex5()
data = pd.read_csv("ex5.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Degree,average
0,graduate,62.50
1,undergraduate,61.63


In [16]:
def ex6():
    # Write an SQL statement that calculates the exam averages for students; sort by average in descending order. Show only top 10 students
    # output columns: first_name, last_name, degree, average
    # round to two decimal places
    
    # YOUR CODE HERE
    sql_statement = "SELECT students.First_Name, students.Last_Name, students.degree, round(avg(StudentExamScores.score),2) as average  from StudentExamScores inner join students on students.studentID = StudentExamScores.studentID  group by students.studentID order by average desc limit 10;"  
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [17]:
sql_statement = ex6()
data = pd.read_csv("ex6.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,First_Name,Last_Name,Degree,average
0,John,Washington,graduate,83.00
1,Robert,Andrade,undergraduate,82.00
2,Paul,Smith,undergraduate,79.00
3,Jason,Delgado,undergraduate,77.50
4,Jason,Thompson,graduate,75.00
5,Calvin,Perez,undergraduate,75.00
6,Calvin,Martinez,undergraduate,74.50
7,Billy,Palmer,undergraduate,74.38
8,Matthew,King,undergraduate,73.56
9,Debra,Pratt,undergraduate,72.60
